This notebook computes the walking times between pairs of voting locations. While this notebook specifically does Los Angeles County, one can similarly do any other region by replacing certain values (marked with !! comments) with desired ones. 

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
from shapely.geometry import MultiPoint
import networkx as nx
import osmnx as ox

Example: Los Angeles County

load data

In [ ]:
#loading street network
place = 'Los Angeles County' #!! replace with desired city here
G = ox.graph_from_place(place,network_type='walk', buffer_dist = 5000, simplify=False)

In [ ]:
#loading polling-site location
polls = gpd.read_file('Los Angeles/lac_NoCatalina_polls.geojson') #!! specify the path/directory of polls dataset
nodes=ox.distance.nearest_nodes(G,polls.longitude,polls.latitude)
walk=np.full((len(lac_polls),len(lac_polls)),0)

multiprocess to speed up computation

In [ ]:
#get pairs to prepare for multiprocess
pairs_list = []
for i in range(len(nodes)):
    for j in range(i+1,len(nodes)):
        pairs_list.append((i,j))

In [ ]:
#define function needed for mp
def walk_dist_compute(pair):
    return nx.shortest_path_length(G,nodes[pair[0]],nodes[pair[1]],weight='length')/1.42

In [ ]:
import multiprocess as mp

pool = mp.Pool(60)
distances_list = pool.map(walk_dist_compute, pairs_list)
pool.close()

In [ ]:
#reassembling matrix
for i in range(len(pairs_list)):
    walk[pairs_list[i][0],pairs_list[i][1]]=distances_list[i]
    walk[pairs_list[i][1],pairs_list[i][0]]=distances_list[i]

In [ ]:
np.save('lac_walk.npy',walk) #name file as desired